##### I have tried forecasting using lightgbm on GOOG data but result were pretty bad the reason may be the forecasting with small amout of close price data will not be enough. I need use multiple feature which have high correlation with dependent variable example volume to decrease Mean-Absolute-Error value.
##### For now I will use AUDUSD 1Hr data since already have tuned models.

In [3]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import pandas as pd
from backtesting.test import SMA, GOOG

data = pd.read_csv("/home/ujwal/ML_program/Trading_financial_market/Algo-trading-with-ML/algo stratergy 1/AUDUSD_Candlestick_1_Hour_BID_03.09.2007-01.10.2022.csv", index_col='Gmt time', parse_dates=True)


In [5]:
class SmaCross(Strategy):
    q=265
    w=145
    def init(self):
        # q=265
        # w=145
        price = self.data.Close
        self.ma1 = self.I(SMA, price, self.q)
        self.ma2 = self.I(SMA, price, self.w)

    def next(self):
        if crossover(self.ma1, self.ma2):
            self.buy(size=10000)
        elif crossover(self.ma2, self.ma1):
            self.sell(size=10000)
df = data.sort_index()
# print(len(df))
# print("train=",len(df)*0.90)
optimize, test = df[:118972], df[118972:] 
print(len(optimize), len(test))


118972 13220


In [ ]:
# bt = Backtest(optimize, SmaCross, cash=10000, commission=0.01, margin=0.05, exclusive_orders=True)
# optimized = bt.optimize(q=range(260,270,1), w=range(140,150,1), maximize='Calmar Ratio')
# optimized
# optimized._strategy
# # SmaCross(q=265,w=145 got calmer Ratio of 3.594713

In [6]:
bt = Backtest(test, SmaCross, cash=10000, commission=0.01, margin=0.05, exclusive_orders=True)
stats = bt.run()
stats
# Something wrong the value looks to good to be true, I think I should split data into 2 part one for optimize another for testing.

Start                     2021-03-30 04:00:00
End                       2022-12-09 23:00:00
Duration                    619 days 19:00:00
Exposure Time [%]                    96.83056
Equity Final [$]                    16913.868
Equity Peak [$]                      17117.48
Return [%]                           69.13868
Buy & Hold Return [%]               -9.918835
Return (Ann.) [%]                   41.643096
Volatility (Ann.) [%]               37.492423
Sharpe Ratio                         1.110707
Sortino Ratio                         2.90269
Calmar Ratio                         4.084082
Max. Drawdown [%]                  -10.196439
Avg. Drawdown [%]                    -0.89628
Max. Drawdown Duration       87 days 01:00:00
Avg. Drawdown Duration        4 days 06:00:00
# Trades                                   62
Win Rate [%]                        61.290323
Best Trade [%]                      10.028343
Worst Trade [%]                     -2.539272
Avg. Trade [%]                    

#### Increasing trade size according to ML confluence.

In [7]:
# 10 day future prediction
# loading model
import pickle
import numpy as np
LGBMRegressor_model = pickle.load(open('/home/ujwal/ML_program/Trading_financial_market/Predicting_AUDUSD_Using_Major_Regression_Models/LGBMRegressor.pkl', 'rb'))
def lgb(prices):
    x = prices.reshape(1,-1)
    forecast = []

    for i in range(5):
        forc = LGBMRegressor_model.predict(X=x)
        forecast = np.append(forecast, forc)
        x = np.append(x, forc)
        x = np.delete(x, 0)
        x = x.reshape(1, -1)
        # print(x)

    return forecast

In [9]:
class SmaCross1(Strategy):
    # q=265
    # w=145
    def init(self):
        q=265
        w=145
        price = self.data.Close
        # a = self._FULL_EQUITY
        # print(a)
        self.ma1 = self.I(SMA, price, q)
        self.ma2 = self.I(SMA, price, w)

    def next(self):
        prices = self.data.Close[-60:]
        eqty = self.equity
        # b= self.buy()
        if crossover(self.ma1, self.ma2):
            # self.buy(size=self._FULL_EQUITY)
            fore_val = lgb(prices) 
            if prices[-1:]<=(fore_val[0] and fore_val[1] and fore_val[2] and fore_val[3] and fore_val[4]):
                  self.buy(size=int(eqty*2))
            elif prices[-1:]<=(fore_val[0] and fore_val[1] and fore_val[2] and fore_val[3]) :
                self.buy(size=int(eqty*1.5))
            elif prices[-1:]<=(fore_val[0] and fore_val[1] and fore_val[2]) :
                self.buy(size=int(eqty*1))
            elif prices[-1:]<=(fore_val[0] and fore_val[1]) :
                self.buy(size=int(eqty))
            elif prices[-1:]<=fore_val[0] :
                self.buy(size=int(eqty*0.5))
                # print(int(a))


        elif crossover(self.ma2, self.ma1):
            fore_val = lgb(prices)     
            if prices[-1:]>=(fore_val[0] and fore_val[1] and fore_val[2] and fore_val[3] and fore_val[4]):
                self.sell(size=int(eqty*2))   
            elif prices[-1:]>=(fore_val[0] and fore_val[1] and fore_val[2] and fore_val[3]) :
                self.sell(size=int(eqty*1.5)) 
            elif prices[-1:]>=(fore_val[0] and fore_val[1] and fore_val[2]) :
                self.sell(size=int(eqty*1))
            elif prices[-1:] >= (fore_val[0] and fore_val[1]) :
                self.sell(size=int(eqty))
            elif prices[-1:]>=fore_val[0] :
                self.sell(size=int(eqty*0.5))
                # print(int(self.equity*0.01))



df = data.sort_index()
# print(len(df))
# print("train=",len(df)*0.90)
optimize, test = df[:118972], df[118972:] 
print(len(optimize), len(test))


118972 13220


In [10]:
bt = Backtest(test, SmaCross1, cash=10000, commission=0.01, margin=0.05, exclusive_orders=True)
stats = bt.run()
stats
# Something wrong the value looks to good to be true, I think I should split data into 2 part one for optimize another for testing.

Start                     2021-03-30 04:00:00
End                       2022-12-09 23:00:00
Duration                    619 days 19:00:00
Exposure Time [%]                    96.83056
Equity Final [$]                 21944.365692
Equity Peak [$]                   22788.94152
Return [%]                         119.443657
Buy & Hold Return [%]               -9.918835
Return (Ann.) [%]                   68.307426
Volatility (Ann.) [%]              114.315297
Sharpe Ratio                         0.597535
Sortino Ratio                        2.022475
Calmar Ratio                         2.909552
Max. Drawdown [%]                  -23.476956
Avg. Drawdown [%]                   -2.478748
Max. Drawdown Duration      120 days 16:00:00
Avg. Drawdown Duration        5 days 12:00:00
# Trades                                   39
Win Rate [%]                        58.974359
Best Trade [%]                      10.028343
Worst Trade [%]                     -3.613742
Avg. Trade [%]                    

#### Lets check for full dataframe.

In [12]:
# LGBMRegressor confluence.
bt = Backtest(df, SmaCross1, cash=10000, commission=0.01, margin=0.05, exclusive_orders=True)
stats = bt.run()
stats
# Something wrong the value looks to good to be true, I think I should split data into 2 part one for optimize another for testing.

Start                     2007-01-10 00:00:00
End                       2022-12-09 23:00:00
Duration                   5812 days 23:00:00
Exposure Time [%]                   99.767762
Equity Final [$]           58830604001.500519
Equity Peak [$]            61094865789.393616
Return [%]                   588305940.015005
Buy & Hold Return [%]              -22.617805
Return (Ann.) [%]                  180.933144
Volatility (Ann.) [%]               457.70381
Sharpe Ratio                         0.395306
Sortino Ratio                        3.238896
Calmar Ratio                         2.781923
Max. Drawdown [%]                  -65.038873
Avg. Drawdown [%]                   -3.114196
Max. Drawdown Duration      391 days 01:00:00
Avg. Drawdown Duration        5 days 19:00:00
# Trades                                  423
Win Rate [%]                        61.702128
Best Trade [%]                      34.107959
Worst Trade [%]                    -23.224998
Avg. Trade [%]                    

In [13]:
# Without confluence.
bt = Backtest(df, SmaCross, cash=10000, commission=0.01, margin=0.05, exclusive_orders=True)
stats = bt.run()
stats
# Something wrong the value looks to good to be true, I think I should split data into 2 part one for optimize another for testing.

Start                     2007-01-10 00:00:00
End                       2022-12-09 23:00:00
Duration                   5812 days 23:00:00
Exposure Time [%]                   99.767762
Equity Final [$]                   181924.872
Equity Peak [$]                    182128.484
Return [%]                         1719.24872
Buy & Hold Return [%]              -22.617805
Return (Ann.) [%]                   21.196361
Volatility (Ann.) [%]               13.444903
Sharpe Ratio                         1.576535
Sortino Ratio                         4.98178
Calmar Ratio                         3.251078
Max. Drawdown [%]                   -6.519795
Avg. Drawdown [%]                   -0.194649
Max. Drawdown Duration      202 days 01:00:00
Avg. Drawdown Duration        3 days 17:00:00
# Trades                                  706
Win Rate [%]                        62.889518
Best Trade [%]                      34.107959
Worst Trade [%]                     -7.757331
Avg. Trade [%]                    